In [1]:
from google.colab import drive
drive.mount('/content/drive')
!nvidia-smi

Mounted at /content/drive
Thu Dec  9 10:52:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                              

In [2]:
cd /content/drive/My Drive/papers/Research/SchemaTranslation/SchemaTranslation

/content/drive/My Drive/papers/Research/SchemaTranslation/SchemaTranslation


In [3]:
# setup
!pip install -r requirement.txt
!rm -rf eval
!unzip eval.zip
!cd eval && bash install_dependencies.sh

     |████████████████████████████████| 65 kB 3.2 MB/s 
     |████████████████████████████████| 3.1 MB 17.0 MB/s 
     |████████████████████████████████| 1.2 MB 59.8 MB/s 
     |████████████████████████████████| 298 kB 77.1 MB/s 
     |████████████████████████████████| 143 kB 64.8 MB/s 
     |████████████████████████████████| 596 kB 69.3 MB/s 
     |████████████████████████████████| 895 kB 65.5 MB/s 
     |████████████████████████████████| 3.3 MB 37.2 MB/s 
     |████████████████████████████████| 61 kB 631 kB/s 
     |████████████████████████████████| 132 kB 73.6 MB/s 
     |████████████████████████████████| 243 kB 75.5 MB/s 
     |████████████████████████████████| 1.1 MB 71.4 MB/s 
     |████████████████████████████████| 192 kB 70.9 MB/s 
     |████████████████████████████████| 271 kB 74.1 MB/s 
     |████████████████████████████████| 160 kB 68.4 MB/s 
  Created wheel for entmax: filename=entmax-1.0-py3-none-any.whl size=11015 sha256=84704864c2e71b745ca0a7942322b07fd533bce0fb734412ad2

In [4]:
# train H2H
pretrained_model="facebook/m2m100_418M"
tgt="fr"
data_format="column-to-column"
max_source_length=128
warmup=0.2
batch_size=4
lr=3e-5
num_epoch=4
model_class="M2M100ForST"
output_suffix=model_class+"-H2H"

# train
!python run_Concat.py \
        --model_name_or_path $pretrained_model \
        --do_train \
        --do_eval \
        --do_predict \
        --source_lang=en \
        --target_lang=$tgt \
        --train_file=data/$data_format/train.jsonl \
        --validation_file=data/$data_format/valid.jsonl \
        --test_file=data/$data_format/test.jsonl \
        --output_dir=outputs/$data_format/$tgt/$output_suffix/ \
        --max_source_length=$max_source_length \
        --per_device_train_batch_size=$batch_size \
        --per_device_eval_batch_size=$batch_size \
        --learning_rate=$lr --warmup_ratio=$warmup --num_train_epochs=$num_epoch  \
        --save_strategy=epoch \
        --evaluation_strategy=epoch \
        --overwrite_output_dir \
        --model_class=$model_class \
        --load_best_model_at_end=True \
        --predict_with_generate

# eval
root_dir="../"
!cd eval && cat "$root_dir"/data/"$data_format"/test_ref".$tgt" | sh tok.sh "$tgt" > \
  "$root_dir"/outputs/"$data_format"/"$tgt"/"$output_suffix"/ref.tok
!cd eval && cat "$root_dir"/outputs/"$data_format"/"$tgt"/"$output_suffix"/test_generations.txt | sh tok.sh "$tgt" > \
  "$root_dir"/outputs/"$data_format"/"$tgt"/"$output_suffix"/hyp.tok
!cd eval && python sacrebleu4.py -tok 'none' --score-only \
  "$root_dir"/outputs/"$data_format"/"$tgt"/"$output_suffix"/ref.tok \
  < "$root_dir"/outputs/"$data_format"/"$tgt"/"$output_suffix"/hyp.tok \
  > "$root_dir"/outputs/"$data_format"/"$tgt"/"$output_suffix"/bleu2
!cd eval && cat "$root_dir"/outputs/"$data_format"/"$tgt"/"$output_suffix"/bleu2

12/09/2021 10:56:33 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
12/09/2021 10:56:33 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=True,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.EPOCH,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
generation_max_length=None,
generation_num_beams=None,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=False,
group_by_length=False,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.

In [5]:
# train H2H+CXT
pretrained_model="facebook/m2m100_418M"
tgt="fr"
data_format="column+schema+val-to-column"
max_source_length=128
warmup=0.2
batch_size=4
lr=3e-5
num_epoch=4
model_class="M2M100ForST"
output_suffix=model_class+"-H2H+CXT"

# train
!python run_Concat.py \
        --model_name_or_path $pretrained_model \
        --do_train \
        --do_eval \
        --do_predict \
        --source_lang=en \
        --target_lang=$tgt \
        --train_file=data/$data_format/train.jsonl \
        --validation_file=data/$data_format/valid.jsonl \
        --test_file=data/$data_format/test.jsonl \
        --output_dir=outputs/$data_format/$tgt/$output_suffix/ \
        --max_source_length=$max_source_length \
        --per_device_train_batch_size=$batch_size \
        --per_device_eval_batch_size=$batch_size \
        --learning_rate=$lr --warmup_ratio=$warmup --num_train_epochs=$num_epoch  \
        --save_strategy=epoch \
        --evaluation_strategy=epoch \
        --overwrite_output_dir \
        --model_class=$model_class \
        --load_best_model_at_end=True \
        --predict_with_generate

# eval
root_dir="../"
!cd eval && cat "$root_dir"/data/"$data_format"/test_ref".$tgt" | sh tok.sh "$tgt" > \
  "$root_dir"/outputs/"$data_format"/"$tgt"/"$output_suffix"/ref.tok
!cd eval && cat "$root_dir"/outputs/"$data_format"/"$tgt"/"$output_suffix"/test_generations.txt | sh tok.sh "$tgt" > \
  "$root_dir"/outputs/"$data_format"/"$tgt"/"$output_suffix"/hyp.tok
!cd eval && python sacrebleu4.py -tok 'none' --score-only \
  "$root_dir"/outputs/"$data_format"/"$tgt"/"$output_suffix"/ref.tok \
  < "$root_dir"/outputs/"$data_format"/"$tgt"/"$output_suffix"/hyp.tok \
  > "$root_dir"/outputs/"$data_format"/"$tgt"/"$output_suffix"/bleu2
!cd eval && cat "$root_dir"/outputs/"$data_format"/"$tgt"/"$output_suffix"/bleu2

12/09/2021 11:37:45 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
12/09/2021 11:37:45 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=True,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.EPOCH,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
generation_max_length=None,
generation_num_beams=None,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=False,
group_by_length=False,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.

In [6]:
# train CAST
pretrained_model="facebook/m2m100_418M"
tgt="fr"
tgt_lang="fr"
data_format="column+schema+val-to-column"
max_source_length=128
warmup=0.2
batch_size=4
lr=3e-5
num_epoch=4
model_class="M2M100RATTypeNStructureSepForST"
output_suffix=model_class+"-CAST"
num_rat_layers=2
encoder_factor=4

# train
!python run_CAST.py \
      --model_name_or_path $pretrained_model \
      --do_train \
      --do_eval \
      --do_predict \
      --source_lang=en \
      --target_lang=$tgt_lang \
      --encoder_factor $encoder_factor \
      --train_file=data/$data_format/train.jsonl \
      --validation_file=data/$data_format/valid.jsonl \
      --test_file=data/$data_format/test.jsonl \
      --output_dir=outputs/$data_format/$tgt/$output_suffix/ \
      --max_source_length=$max_source_length \
      --per_device_train_batch_size=$batch_size \
      --per_device_eval_batch_size=$batch_size \
      --learning_rate=$lr --warmup_ratio=$warmup --num_train_epochs=$num_epoch  \
      --save_strategy=epoch --evaluation_strategy=epoch \
      --overwrite_output_dir \
      --model_class=$model_class --num_rat_layers $num_rat_layers \
      --load_best_model_at_end=True \
      --predict_with_generate

# eval
root_dir="../"
!cd eval && cat "$root_dir"/data/"$data_format"/test_ref".$tgt" | sh tok.sh "$tgt" > \
  "$root_dir"/outputs/"$data_format"/"$tgt"/"$output_suffix"/ref.tok
!cd eval && cat "$root_dir"/outputs/"$data_format"/"$tgt"/"$output_suffix"/test_generations.txt | sh tok.sh "$tgt" > \
  "$root_dir"/outputs/"$data_format"/"$tgt"/"$output_suffix"/hyp.tok
!cd eval && python sacrebleu4.py -tok 'none' --score-only \
  "$root_dir"/outputs/"$data_format"/"$tgt"/"$output_suffix"/ref.tok \
  < "$root_dir"/outputs/"$data_format"/"$tgt"/"$output_suffix"/hyp.tok \
  > "$root_dir"/outputs/"$data_format"/"$tgt"/"$output_suffix"/bleu2
!cd eval && cat "$root_dir"/outputs/"$data_format"/"$tgt"/"$output_suffix"/bleu2

12/09/2021 12:25:49 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
12/09/2021 12:25:49 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=True,
do_train=True,
eval_accumulation_steps=None,
eval_steps=None,
evaluation_strategy=IntervalStrategy.EPOCH,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
generation_max_length=None,
generation_num_beams=None,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=False,
group_by_length=False,
hub_model_id=None,
hub_strategy=HubStrategy.EVERY_SAVE,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
label_names=None,
label_smoothing_factor=0.